In [2]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


## Video transcription
@forusone (shins777@gmail.com)

### Video transcription and translation

This colab used YT videos to test Gemini's transciption and translation.

### Install Vertex AI SDK for Python

In [3]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    google-cloud-speech \
    google-cloud-storage \
    google-cloud-videointelligence

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.2/324.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.8/269.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00


In [4]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

    from google.colab import drive
    drive.mount('/content/drive')

!gcloud config set project ai-hangsik


Mounted at /content/drive
Updated property [core/project].


### Initial set up

In [5]:
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
MODEL = "gemini-1.5-pro-002"

import base64
import vertexai
from vertexai.generative_models import GenerativeModel

vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel(MODEL)

## Transcript and translation

### Helper functions

In [6]:

def generate(prompt:str,
             video_uri:str,
             response_schema)->str:
  """
  Function to call Gemini for Transcription and Translation.

  Args:
    prompt: Prompt to instruct transcription and translation.
    yt_uri: Youtube video URI to analyze.

  Returns:
    A transcription or translation with specified response schema.(JSON)

  """

  from vertexai.generative_models import (
        Part,
        SafetySetting,
        GenerationConfig
  )

  # Configuration to contorl LLM generation.
  generation_config = GenerationConfig(
      max_output_tokens = 8192,
      temperature=1,
      top_p =0.95,
      response_mime_type="application/json",
      response_schema=response_schema
  )

  # Sate setting
  safety_settings = [
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
  ]

  video1 = Part.from_uri(
      mime_type="video/*",
      uri=video_uri,
  )

  responses = model.generate_content(
      [prompt, video1],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=False,
  )

  return responses.text


### YT Video list

In [7]:

if True: # Videos in YT platform

  # video_uri="https://www.youtube.com/watch?v=PNTCM7cbrsc" # Small Talk
  # video_uri="https://www.youtube.com/watch?v=wrh-4GZN3aE" # Dad Won't Let Daughter Marry For Love
  # video_uri = "https://www.youtube.com/watch?v=CYyUuIXzGgI" # Game Theory Scene | 21(2008)
  # video_uri = "https://www.youtube.com/watch?v=OoUVSHDbAeM" # Stephen Hawking Discovers The Black Hole Theory
  # video_uri = "https://www.youtube.com/watch?v=Gth02sjU4wU" # [단편영화] A (2017)
  # video_uri = "https://www.youtube.com/watch?v=lSeBy_lqs28" # 이 뭔 개소리야 긴버전 (원본영상)
  # video_uri = "https://www.youtube.com/watch?v=IuiCRoz2gMY" # 누구인가? 누가 기침 소리를 내었는가 -궁예-
  video_uri = "https://www.youtube.com/watch?v=ydmF-WoL59s" # 강동원 궁중악사(얼쑤얼쑤 하는거)

else: # videos in GCS
  # video_uri="gs://tests_nov25_2024/translation/small_talk.mp4"  # Small Talk
  video_uri="gs://tests_nov25_2024/translation/Stephen Hawking Discovers The Black Hole Theory | The Theory Of Everything (2014) | Screen Bites.mp4" # Stephen Hawking Discovers The Black Hole Theory




### Video transcription

In [8]:
import json

prompt_transcript = """
당신을 비디오를 분석해서 transcript를 작성해야 하는 AI Assistant 입니다.
아래 가이드라인에 맞게 transcription을 작성해주세요.

1. 첨부된 비디오를 분석하여 아래와 같은 포맷으로 반드시 모든 대화 내용을 하나도 빠짐없이 모두 출력해주세요.
2. 결과 출력 단위는 비디오 내의 장면이 구분되는 특정 장소를 기준으로 나누어서 출력해주세요.
3. 목소리를 기반으로 화자(speaker)를 정확하게 분리해서 출력해주세요.
4. 목소리외에 다양한 효과음, 감정표현은 괄호를 사용해서 반드시 자세히 표현해주세요.

"""

# Response schema can control the output of generation from Gemini.
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "location": { "type": "STRING",},
            "start_time": { "type": "STRING",},
            "end_time": { "type": "STRING",},
            "elapsed_time": { "type": "STRING",},
            "transcription": {
                "type": "ARRAY",
                "items" : {
                  "type": "OBJECT",
                  "properties": {
                    "speaker": { "type": "STRING",},
                    "transcript": { "type": "STRING",},
                  }
                }
            },
        },
        "required": ["start_time","end_time","transcription"],
    },
}

json_str = generate(prompt_transcript, video_uri, response_schema)
json_transcript = json.loads(json_str)


In [9]:
for transcription in json_transcript:
  print(transcription)
  print("-"*20)

{'location': '궁궐', 'start_time': '00:00:06', 'end_time': '00:00:11', 'elapsed_time': '00:00:05', 'transcription': [{'speaker': '신하', 'transcript': '옥황상제님의 아드님께서 오신다! 예를 갖춰라!'}]}
--------------------
{'location': '궁궐', 'start_time': '00:00:19', 'end_time': '00:00:24', 'elapsed_time': '00:00:05', 'transcription': [{'speaker': '옥황상제 아들', 'transcript': '지상의 왕은 황금 1만냥을 함경도 기근 지역에 보냈느냐?'}]}
--------------------
{'location': '궁궐', 'start_time': '00:00:24', 'end_time': '00:00:30', 'elapsed_time': '00:00:06', 'transcription': [{'speaker': '왕', 'transcript': '아, 예. 그제 제 꿈에 나타나 하명하신대로 한 치 틀림없이 그리 하였습니다.'}]}
--------------------
{'location': '궁궐', 'start_time': '00:00:30', 'end_time': '00:00:38', 'elapsed_time': '00:00:08', 'transcription': [{'speaker': '옥황상제 아들', 'transcript': '하늘에서 그대의 덕을 높이사 그대가 하늘로 돌아올 때 7배, 70배, 700배로 갚아줄 것이다.'}]}
--------------------
{'location': '궁궐', 'start_time': '00:00:38', 'end_time': '00:00:42', 'elapsed_time': '00:00:04', 'transcription': [{'speaker': '왕', 'transcript

### Video Translation

In [25]:
languages = ["영어","브라질 포르투갈어","독일어","스페인어","프랑스어","이탈리아어","말레이시아어","태국어"]
language = languages[1]
print(language)

브라질 포르투갈어


In [23]:
prompt_translation = f"""
당신을 원본 언어로 된 문서를 기반으로 비디오를 분석해서 {language}로 번역을 해야 하는 AI Assistant 입니다.
반드시 비디오의 모든 장면에 나타난 모든 대화 내용을 아래 가이드라인에 맞게 정확하게 번역 해주세요.

1. 번역을 할 때는 첨부된 json 내에서 transcription 부분만 번역 해주세요.
2. 번역결과는 할 때는 json구조를 그대로 유지해서 표현해주세요.
3. 목소리외에 다양한 효과음, 감정표현은 괄호를 사용해서 반드시 자세히 표현해주세요.

원본 문서 : {json_transcript}

"""

json_str = generate(prompt_translation,video_uri, response_schema) # Used same response schema with transcription process.
json_translation = json.loads(json_str)


SyntaxError: '[' was never closed (<ipython-input-23-3031d807b242>, line 1)

In [18]:
for transcription, translation in zip( json_transcript, json_translation):
  print(transcription)
  print(translation)
  print("-"*20)


{'location': '궁궐', 'start_time': '00:00:06', 'end_time': '00:00:11', 'elapsed_time': '00:00:05', 'transcription': [{'speaker': '신하', 'transcript': '옥황상제님의 아드님께서 오신다! 예를 갖춰라!'}]}
{'location': 'Palácio', 'start_time': '00:00:06', 'end_time': '00:00:11', 'elapsed_time': '00:00:05', 'transcription': [{'speaker': 'Servo', 'transcript': 'O filho do Imperador de Jade está vindo! Preparem-se para mostrar respeito!'}]}
--------------------
{'location': '궁궐', 'start_time': '00:00:19', 'end_time': '00:00:24', 'elapsed_time': '00:00:05', 'transcription': [{'speaker': '옥황상제 아들', 'transcript': '지상의 왕은 황금 1만냥을 함경도 기근 지역에 보냈느냐?'}]}
{'location': 'Palácio', 'start_time': '00:00:19', 'end_time': '00:00:24', 'elapsed_time': '00:00:05', 'transcription': [{'speaker': 'Filho do Imperador de Jade', 'transcript': 'O rei terreno enviou os 10.000 nyang de ouro para a área de fome em Hamgyong?'}]}
--------------------
{'location': '궁궐', 'start_time': '00:00:24', 'end_time': '00:00:30', 'elapsed_time': '00:00:06',

### Compare transcript and translation

In [19]:
from IPython.display import display, Markdown
import pandas as pd

df = pd.DataFrame()

for transcript, translation in zip( json_transcript, json_translation):

  for script, trans in zip(transcript['transcription'], translation['transcription']):
    script_df = pd.DataFrame.from_dict(script, orient='index').T
    trans_df = pd.DataFrame.from_dict(trans, orient='index').T

    script_df['translation'] = trans_df['transcript']

    df = pd.concat([df, script_df], ignore_index=True)  # Concatenate script_df to df

df

,speaker,transcript,translation
0,신하,옥황상제님의 아드님께서 오신다! 예를 갖춰라!,O filho do Imperador de Jade está vindo! Prepa...
1,옥황상제 아들,지상의 왕은 황금 1만냥을 함경도 기근 지역에 보냈느냐?,O rei terreno enviou os 10.000 nyang de ouro p...
2,왕,"아, 예. 그제 제 꿈에 나타나 하명하신대로 한 치 틀림없이 그리 하였습니다.","Ah, sim. Exatamente como você ordenou em meu s..."
3,옥황상제 아들,"하늘에서 그대의 덕을 높이사 그대가 하늘로 돌아올 때 7배, 70배, 700배로 갚...",O céu aprecia sua virtude e a recompensará set...
4,왕,왕가의 보물을 보자시기에 그건 역시 준비했습니다.,Como você queria ver os tesouros da família re...
5,옥황상제 아들,지상의 왕이 보기보다 아주 똘똘하구나.,O rei terreno é mais inteligente do que parece.
6,왕,예.,Sim.
7,옥황상제 아들,헌데 궁녀들은 어디에?,Mas onde estão as damas da corte?
8,왕,궁녀들은 어찌하여 그러시는지.,Por que você pergunta sobre as damas da corte?
9,옥황상제 아들,너는 연불한다고 궁녀들이 술 좀 따라주면 좋겠는데.,Eu gostaria que as damas da corte me servissem...


### Write the result into a file in drive

In [20]:
cd "/content/drive/MyDrive/Colab Notebooks/projects/TVPlus_Translation/notebook/translation"

/content/drive/MyDrive/Colab Notebooks/projects/TVPlus_Translation/notebook/translation


In [21]:
!pwd

/content/drive/MyDrive/Colab Notebooks/projects/TVPlus_Translation/notebook/translation


In [22]:
df.to_csv("./transcript_translation_br.csv", index=True)